In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Limpieza de los datos



1. Cargar los datos

In [27]:
calificaciones = pd.read_csv("./Datos/calificaciones.csv")
estudiantes = pd.read_csv("./Datos/estudiantes.csv")
carreras = pd.read_csv("./Datos/carrera.csv")
escuelas = pd.read_csv("./Datos/escuelas.csv")

### Primero eliminamos a aquellos estudiantes que su "prepa" no esta registrada como escuela

con esto se quitaron aprox. 2000-3000 filas

In [28]:
ciud = pd.DataFrame(estudiantes["ciudad"].unique(), columns=["ciudad"])
#ciud

In [29]:
#colonias = pd.DataFrame(estudiantes["municipio"].unique(), columns=["municipio"])
#colonias

#municipio_ciudad = estudiantes.groupby('municipio')['ciudad'].unique().reset_index()
#municipio_ciudad.columns = ['municipio', 'ciudades']
#municipio_ciudad

#if estudiantes["municipio"] > 25000 && estudiantes["municipio"] < 25019:
#    estudiantes["ciudad"]


correcta = pd.merge(estudiantes, escuelas[['clave']], left_on='prepa', right_on='clave', how='inner')
#correcta.sort_values('clave_alumno')



### Ahora se filtran todas las calificaciones de alumnos, si la clave del alumno no esta registrada en alumnos se descartan.

In [30]:
correcta2 = pd.merge(calificaciones, estudiantes[['clave_alumno']], left_on='clave_alumno', right_on='clave_alumno', how='inner')
#correcta2.sort_values('clave_alumno')

### aqui filtro a aquellos estudiantes que necesitamos solamente, aquellos que siguen vigentes y que no han terminado sus estudios

In [31]:
correcta3 = estudiantes[(estudiantes['termino'].isnull()) & (estudiantes['causa_salida']==1)]
#correcta3.sort_values('clave_alumno')

In [32]:
nombres_unicos = pd.DataFrame(estudiantes.groupby("municipio")["ciudad"].apply(lambda x: x.unique()))
#nombres_unicos
#nombres_unicos.to_csv('nombres.csv', index=True)


### Aqui ponemos la clave de los municipios correctos, si encuentra otra clave pone por default la de culiacan

In [33]:
#los municipios con clave le asignamos su ciudad correspondiente
correcta3['ciudad'] = correcta3['municipio'].apply(lambda x: 'Los Mochis' if x == 25001 else 'Angostura' if x == 25002 else 'Badiraguato' if x == 25003 else 'Concordia' if x == 25004 else 'Cosala' if x == 25005 else 'Culiacan' if x == 25006 else 'Choix' if x == 25007 else 'La Cruz' if x == 25008 else 'Escuinapa' if x == 25009 else 'El Fuerte' if x == 25010 else 'Guasave' if x == 25011 else 'Mazatlan' if x == 25012 else 'Mocorito' if x == 25013 else 'Rosario' if x == 25014 else 'Guamuchil' if x == 25015 else 'San Ignacio' if x == 25016 else 'Sinaloa de Leyva' if x == 25017 else 'Navolato' if x == 25018 else 'ElDorado' if x == 25019 else 'Juan Jose Rios' if x == 25020 else 'No encontrado')
#para los municipios que no tienen bien su clave se le asignan por default a culiacan
clave_municipios = [25001, 25002, 25003, 25004, 25005, 25006, 25007, 25008, 25009, 25010, 25011, 25012, 25013, 25014, 25015, 25016, 25017, 25018, 25019, 25020]
clave_cul = 25006
#si no esta en la lista de claves de municipios se le asigna a culiacan
correcta3.loc[~correcta3['municipio'].isin(clave_municipios), 'ciudad'] = 'Culiacan'
#ahora cambiamos los municipios que estan mal a la clave de culiacan
correcta3['municipio'] = correcta3['municipio'].apply(lambda x: clave_cul if x not in clave_municipios else x)

C:\Users\mari_\AppData\Local\Temp\ipykernel_6256\4072386065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correcta3['ciudad'] = correcta3['municipio'].apply(lambda x: 'Los Mochis' if x == 25001 else 'Angostura' if x == 25002 else 'Badiraguato' if x == 25003 else 'Concordia' if x == 25004 else 'Cosala' if x == 25005 else 'Culiacan' if x == 25006 else 'Choix' if x == 25007 else 'La Cruz' if x == 25008 else 'Escuinapa' if x == 25009 else 'El Fuerte' if x == 25010 else 'Guasave' if x == 25011 else 'Mazatlan' if x == 25012 else 'Mocorito' if x == 25013 else 'Rosario' if x == 25014 else 'Guamuchil' if x == 25015 else 'San Ignacio' if x == 25016 else 'Sinaloa de Leyva' if x == 25017 else 'Navolato' if x == 25018 else 'ElDorado' if x == 25019 else 

### Para limpiar los textos que tengan mal el nombre de municipio

In [34]:
from fuzzywuzzy import fuzz

# Función que compara una cadena con "culiacan"
def buscar_culiacan_fuzzy(texto):
    # Asegúrate de que el texto no sea NaN ni nulo antes de aplicar fuzzy matching
    if isinstance(texto, str) and len(texto) > 0:
        if fuzz.ratio(texto.lower(), "culiacan") > 40:  # Umbral de coincidencia
            return 'culiacan'
        elif fuzz.ratio(texto.lower(), "aguaruto") > 40:  # Umbral de coincidencia
            return 'aguaruto'
        elif fuzz.ratio(texto.lower(), "angostura") > 40:  # Umbral de coincidencia
            return 'angostura'
        elif fuzz.ratio(texto.lower(), "bachimeto") > 40:  # Umbral de coincidencia
            return 'bachimeto'
        elif fuzz.ratio(texto.lower(), "concordia") > 40:  # Umbral de coincidencia
            return 'concordia'
        elif fuzz.ratio(texto.lower(), "dorado") > 40:  # Umbral de coincidencia
            return 'el dorado'
        elif fuzz.ratio(texto.lower(), "guamuchil") > 40:  # Umbral de coincidencia
            return 'guamuchil'
        elif fuzz.ratio(texto.lower(), "guasave") > 40:  # Umbral de coincidencia
            return 'guasave'
        elif fuzz.ratio(texto.lower(), "villa juarez") > 40:  # Umbral de coincidencia
            return 'villa juarez'
        elif fuzz.ratio(texto.lower(), "pericos") > 40:  # Umbral de coincidencia
            return 'pericos'
        elif fuzz.ratio(texto.lower(), "salvador alvarado") > 40:  # Umbral de coincidencia
            return 'salvador alvarado'
        elif fuzz.ratio(texto.lower(), "mazatlan") > 40:  # Umbral de coincidencia
            return 'mazatlan'
                   
    return texto

# Aplicar la función a la columna 'ciudad'
df = correcta3
df.loc[:, 'ciudad'] = correcta3['ciudad'].apply(buscar_culiacan_fuzzy)

# Ver el resultado
#correcta3



### Aqui limpiamos algunas colonias que tienen problemas con el nombre

In [35]:
correcta3['colonia'] = correcta3['colonia'].replace({
    '21': '21 DE MARZO', 
    '21DE': '21 DE MARZO',
    'ADF': 'ADOLFO LOPEZ MATEOS',
    'ADOLFO': 'ADOLFO LOPEZ MATEOS',
    'AF': 'ADOLFO LOPEZ MATEOS',
    'AGRI.': 'COLONIA AGRICOLA MÉXICO',
    'AGRICOLA': 'COLONIA AGRICOLA MÉXICO',
    'COL.AGRICOLA': 'COLONIA AGRICOLA MÉXICO',
    'AGUA': 'AGUACALIENTE',
    'AGUA CALIENTE': 'AGUACALIENTE',
    'ALFONSO': 'ALFONSO G. CALDERON',
    'AMP.TOLEDO': 'AMP. TOLEDO',
    'ALTURAS': 'ALTURAS DEL SUR',
    'AMADO': 'AMADO NERVO',
    'AQUILES': 'AQUILES SERDAN',
    'BALCONES': 'BALCONES DE NUEVO CULIACAN',
    'BUENOS': 'BUENOS AIRES',
    'BUENA': 'BUENA VISTA',
    'CNOP': 'AMP. CNOP',
    'AMPL.CNOP': 'AMP. CNOP',
    'CU': 'CD. UNIVERSITARIA',
    'C.U': 'CD. UNIVERSITARIA',
    'CIUDADES': 'CIUDADES HERMANAS',
    'CD.': 'CIUDADES HERMANAS',
    'CD. HERMANAS': 'CIUDADES HERMANAS',
    'CHULA': 'CHULAVISTA',
    'CHULA VISTA': 'CHULAVISTA',
    'CONOCIDO': 'CONOCIDA',
    'CONOSIDO': 'CONOCIDA',
    'CONOSIDA': 'CONOCIDA',
    'FRAC.NAKAYAMA': 'FRACC. NAKAYAMA',
    'FRAC.': 'FRACC.',
    'FRA': 'FRACC.',
    'FRACC.-': 'FRACC.',
    'EJ.': 'EJIDAL',
    'E.': 'EJIDAL',
    'CANACO': 'FRACC. CANACO',
    'FRACC.CANACO': 'FRACC. CANACO',
    'ESCOBEDO': 'COL. MARIANO ESCOBEDO',
    'ESCOBESO': 'COL. MARIANO ESCOBEDO',
    'FDEL GUTIERREZ': 'FIDEL GUTIERREZ',
    'ZAPATA': 'EMILIANO ZAPATA',
    'GENARO': 'COL. GENARO ESTRADA',
    'GUADALUPE': 'GUADALUPE VICTORIA',
    'GPE.': 'GUADALUPE VICTORIA'
})


C:\Users\mari_\AppData\Local\Temp\ipykernel_6256\4193294602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correcta3['colonia'] = correcta3['colonia'].replace({


### vuelvo a filtrar solo las calificaciones de los estudisntes que cumplen las condiciones requeridas

In [36]:
correcta4 = pd.merge(calificaciones, correcta3[['clave_alumno']], left_on='clave_alumno', right_on='clave_alumno', how='inner')
correcta4.sort_values('clave_alumno')


correcta3

,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida
607,3327,4,1,121,BUENOS AIRES,25006.0,culiacan,2043.0,NaN,1
648,4420,4,1,154,El Walamo,25011.0,guasave,2053.0,NaN,1
662,4434,4,1,127,Alcanfores,25018.0,aguaruto,2053.0,NaN,1
668,4440,4,1,117,Los Angeles,25006.0,culiacan,2053.0,NaN,1
671,4443,4,1,142,NaN,25002.0,angostura,2053.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...
11930,11738,8,2,98,Fovissste Playa Azul,25006.0,culiacan,2103.0,NaN,1
11931,11739,8,1,91,LA,25006.0,culiacan,2103.0,NaN,1
11932,11740,8,1,71,BALCONES DE NUEVO CULIACAN,25006.0,culiacan,2103.0,NaN,1
11933,11825,8,1,26,San Miguel,25006.0,culiacan,2103.0,NaN,1


### Eliminamos las colonias que tienen tres o menos caracteres en el dataset

In [37]:
# Eliminar las colonias que tienes 3 o menos caracteres en el dataset
filtro = correcta3['colonia'].dropna().apply(len).between(1, 3)
filtro = filtro.reindex(correcta3.index, fill_value=False)
correcta3 = correcta3[~filtro]
correcta3.reset_index(drop=True, inplace=True)
#correcta3



### Asignamos el nombre de la ciudad o cabecera principal a los municipios con una clave especifica de dicho municipio

In [38]:
#los municipios con clave le asignamos su ciudad correspondiente
correcta3['ciudad'] = correcta3['municipio'].apply(lambda x: 'Los Mochis' if x == 25001 else 'Angostura' if x == 25002 else 'Badiraguato' if x == 25003 else 'Concordia' if x == 25004 else 'Cosala' if x == 25005 else 'Culiacan' if x == 25006 else 'Choix' if x == 25007 else 'La Cruz' if x == 25008 else 'Escuinapa' if x == 25009 else 'El Fuerte' if x == 25010 else 'Guasave' if x == 25011 else 'Mazatlan' if x == 25012 else 'Mocorito' if x == 25013 else 'Rosario' if x == 25014 else 'Guamuchil' if x == 25015 else 'San Ignacio' if x == 25016 else 'Sinaloa de Leyva' if x == 25017 else 'Navolato' if x == 25018 else 'ElDorado' if x == 25019 else 'Juan Jose Rios' if x == 25020 else 'No encontrado')
#para los municipios que no tienen bien su clave se le asignan por default a culiacan
clave_municipios = [25001, 25002, 25003, 25004, 25005, 25006, 25007, 25008, 25009, 25010, 25011, 25012, 25013, 25014, 25015, 25016, 25017, 25018, 25019, 25020]
clave_cul = 25006
#si no esta en la lista de claves de municipios se le asigna a culiacan
correcta3.loc[~correcta3['municipio'].isin(clave_municipios), 'ciudad'] = 'Culiacan'
#ahora cambiamos los municipios que estan mal a la clave de culiacan
correcta3['municipio'] = correcta3['municipio'].apply(lambda x: clave_cul if x not in clave_municipios else x)

C:\Users\mari_\AppData\Local\Temp\ipykernel_6256\4072386065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correcta3['ciudad'] = correcta3['municipio'].apply(lambda x: 'Los Mochis' if x == 25001 else 'Angostura' if x == 25002 else 'Badiraguato' if x == 25003 else 'Concordia' if x == 25004 else 'Cosala' if x == 25005 else 'Culiacan' if x == 25006 else 'Choix' if x == 25007 else 'La Cruz' if x == 25008 else 'Escuinapa' if x == 25009 else 'El Fuerte' if x == 25010 else 'Guasave' if x == 25011 else 'Mazatlan' if x == 25012 else 'Mocorito' if x == 25013 else 'Rosario' if x == 25014 else 'Guamuchil' if x == 25015 else 'San Ignacio' if x == 25016 else 'Sinaloa de Leyva' if x == 25017 else 'Navolato' if x == 25018 else 'ElDorado' if x == 25019 else 

### Combinamos el excel de carreras con sus respectivos alumnos en correcta3

In [39]:
#usando el excel correcta3 y el excel de carreras, pondremos en correcta3 una columna del nombre de la carrera con
#respecto a la clave de la carrera
#correcta3 = pd.merge(correcta3, carreras[['clave_carrera', 'nombre']], left_on='clave_carrera', right_on='clave_carrera', how='left')
#correcta3.rename(columns={'nombre': 'carrera'}, inplace=True)
#correcta3
#correcta3.to_csv('correcta.csv', index=False)

In [40]:
nuevo = pd.merge(correcta3,carreras,how='inner',on='clave_carrera')
nuevo

,clave_alumno,clave_carrera,sexo,prepa,colonia,municipio,ciudad,ingreso,termino,causa_salida,nombre,corto
0,3327,4,1,121,BUENOS AIRES,25006.0,Culiacan,2043.0,NaN,1,INGENIERIA ELECTRONICA,Electronica
1,4420,4,1,154,El Walamo,25011.0,Guasave,2053.0,NaN,1,INGENIERIA ELECTRONICA,Electronica
2,4434,4,1,127,Alcanfores,25018.0,Navolato,2053.0,NaN,1,INGENIERIA ELECTRONICA,Electronica
3,4440,4,1,117,Los Angeles,25006.0,Culiacan,2053.0,NaN,1,INGENIERIA ELECTRONICA,Electronica
4,4443,4,1,142,NaN,25002.0,Angostura,2053.0,NaN,1,INGENIERIA ELECTRONICA,Electronica
...,...,...,...,...,...,...,...,...,...,...,...,...
3672,11737,8,1,153,San Joaquin,25006.0,Culiacan,2103.0,NaN,1,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC
3673,11738,8,2,98,Fovissste Playa Azul,25006.0,Culiacan,2103.0,NaN,1,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC
3674,11740,8,1,71,BALCONES DE NUEVO CULIACAN,25006.0,Culiacan,2103.0,NaN,1,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC
3675,11825,8,1,26,San Miguel,25006.0,Culiacan,2103.0,NaN,1,INGENIERIA EN TECNOLOGIAS DE LA INFORMACION Y ...,TIC
